# Analyze data using ilastik
The notebook shows how load an image from IDR
analyze it in ilastik.
Binary data are read from S3.
The order might need to be adjusted depending on the ilastik project.

### Insert required packages

In [28]:
from os.path import expanduser
import os
import numpy
import zarr
import dask.array as da
import dask.array.image
import s3fs

import omero.clients
from omero.gateway import BlitzGateway
from getpass import getpass
from collections import OrderedDict

import ilastik_main
from ilastik.applets.dataSelection import DatasetInfo
from ilastik.applets.dataSelection.opDataSelection import PreloadedArrayDatasetInfo

# package for 3d visualization
from itkwidgets import view

### Create a connection to IDR

In [29]:
HOST = 'wss://idr.openmicroscopy.org/omero-ws'
conn = BlitzGateway('public', 'public',
                    host=HOST, secure=True)
conn.connect()

True

### Enter the image ID

In [30]:
image_id = 6001240

### Helper function: load an Image as 5D-numpy array: order TZYXC

In [31]:
def load_from_s3(id, resolution='0'):
    cache_size_mb = 2048
    cfg = {
        'anon': True,
        'client_kwargs': {
            'endpoint_url': 'https://minio-dev.openmicroscopy.org/',
        },
        'root': 'idr/zarr/v0.1/%s.zarr/%s/' % (id, resolution)
    }
    s3 = s3fs.S3FileSystem(
        anon=cfg['anon'],
        client_kwargs=cfg['client_kwargs'],
    )
    store = s3fs.S3Map(root=cfg['root'], s3=s3, check=False)
    cached_store = zarr.LRUStoreCache(store, max_size=(cache_size_mb * 2**20))
    # data.shape is (t, c, z, y, x) by convention
    data = da.from_zarr(cached_store)
    values = data[:]
    return numpy.asarray(values)

### Load data

In [32]:
input_data = load_from_s3(image_id)

### View data

In [33]:
values = input_data[0, 0, :, : ,:]
view(values, shadow=False, gradient_opacity=0.4, ui_collapsed=True)

Viewer(gradient_opacity=0.4, rendered_image=<itkImagePython.itkImageUS3; proxy of <Swig Object of type 'itkIma…

### Load each image as an 5D-numpy array and analyze.
Save the probabilities as Zarr zip and link it to the image.

In [34]:
# Load the model linked to the dataset
home = expanduser("~")
model_file = home+"/notebooks/pipelines/pixel-class-133.ilp"
input_data = input_data.swapaxes(1, 2).swapaxes(2, 3).swapaxes(3, 4)
print(input_data.shape)
# Prepare ilastik
os.environ["LAZYFLOW_THREADS"] = "2"
os.environ["LAZYFLOW_TOTAL_RAM_MB"] = "2000"
args = ilastik_main.parse_args([])
args.headless = True
args.project = model_file
shell = ilastik_main.main(args)

print('running ilastik using %s' % model_file)
role_data_dict = OrderedDict(
[
    (
        "Raw Data",
        [
            PreloadedArrayDatasetInfo(preloaded_array=input_data)
        ],
    )
])
predictions = shell.workflow.batchProcessingApplet.run_export(role_data_dict, export_to_array=True)
for data in predictions:
    # Re-organise array from tzyxc to zctyx order expected by OMERO
    data = data.swapaxes(0, 1).swapaxes(3, 4).swapaxes(2, 3).swapaxes(1, 2)
print("done")

(1, 236, 275, 271, 2)
INFO ilastik_main: Starting ilastik from "/srv/conda/envs/notebook".
INFO ilastik_main: Starting ilastik from "/srv/conda/envs/notebook".
INFO ilastik_main: Starting ilastik from "/srv/conda/envs/notebook".
INFO ilastik_main: Starting ilastik from "/srv/conda/envs/notebook".


INFO:ilastik_main:Starting ilastik from "/srv/conda/envs/notebook".


Starting ilastik from "/srv/conda/envs/notebook".
INFO ilastik_main: Resetting lazyflow thread pool with 2 threads.
INFO ilastik_main: Resetting lazyflow thread pool with 2 threads.
INFO ilastik_main: Resetting lazyflow thread pool with 2 threads.
INFO ilastik_main: Resetting lazyflow thread pool with 2 threads.


INFO:ilastik_main:Resetting lazyflow thread pool with 2 threads.


INFO ilastik_main: Configuring lazyflow RAM limit to 2.0GiB
INFO ilastik_main: Configuring lazyflow RAM limit to 2.0GiB
INFO ilastik_main: Configuring lazyflow RAM limit to 2.0GiB
INFO ilastik_main: Configuring lazyflow RAM limit to 2.0GiB


...

DEBUG lazyflow.operators.classifierOperators: Features took 2.660094 seconds. Prediction took 1.6399620000000001 seconds. Subregion: start '[184, 184, 92, 0]' stop '[236, 275, 184, 2]'
DEBUG lazyflow.operators.classifierOperators: Features took 2.660094 seconds. Prediction took 1.6399620000000001 seconds. Subregion: start '[184, 184, 92, 0]' stop '[236, 275, 184, 2]'
DEBUG lazyflow.operators.classifierOperators: Features took 2.660094 seconds. Prediction took 1.6399620000000001 seconds. Subregion: start '[184, 184, 92, 0]' stop '[236, 275, 184, 2]'
DEBUG lazyflow.operators.classifierOperators: Features took 2.660094 seconds. Prediction took 1.6399620000000001 seconds. Subregion: start '[184, 184, 92, 0]' stop '[236, 275, 184, 2]'


DEBUG:lazyflow.operators.classifierOperators:Features took 2.660094 seconds. Prediction took 1.6399620000000001 seconds. Subregion: start '[184, 184, 92, 0]' stop '[236, 275, 184, 2]'


DEBUG lazyflow.operators.classifierOperators: Features took 2.916712 seconds. Prediction took 0.636036 seconds. Subregion: start '[184, 184, 184, 0]' stop '[236, 275, 271, 2]'
DEBUG lazyflow.operators.classifierOperators: Features took 2.916712 seconds. Prediction took 0.636036 seconds. Subregion: start '[184, 184, 184, 0]' stop '[236, 275, 271, 2]'
DEBUG lazyflow.operators.classifierOperators: Features took 2.916712 seconds. Prediction took 0.636036 seconds. Subregion: start '[184, 184, 184, 0]' stop '[236, 275, 271, 2]'
DEBUG lazyflow.operators.classifierOperators: Features took 2.916712 seconds. Prediction took 0.636036 seconds. Subregion: start '[184, 184, 184, 0]' stop '[236, 275, 271, 2]'


DEBUG:lazyflow.operators.classifierOperators:Features took 2.916712 seconds. Prediction took 0.636036 seconds. Subregion: start '[184, 184, 184, 0]' stop '[236, 275, 271, 2]'


done


### View the results.

In [35]:
data_viewer = data[:, 0, 0, :, :]
data_viewer = data_viewer.astype(numpy.uint8)
view(data_viewer)

Viewer(rendered_image=<itkImagePython.itkImageUC3; proxy of <Swig Object of type 'itkImageUC3 *' at 0x7f775590…

### Close the connection

In [36]:
conn.close()

### License
Copyright (C) 2019-2020 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.